In [1]:
# Initialize
import numpy as np
import pandas as pd
import re

In [2]:
peptides_file = 'pd22_exports/pd22_export_exp3_Heart_DKO-TAC-Comparison_PeptideGroups.txt'
proteins_file = 'pd22_exports/pd22_export_exp3_Heart_DKO-TAC-Comparison_Proteins.txt'
pep = pd.read_csv(peptides_file, sep='\t', low_memory=False)
pro = pd.read_csv(proteins_file, sep='\t', low_memory=False)

# Import data from proteome discoverer

For the peptides dataframe, use the first listed Master Protein Accession ID as the primary identifier. This identifier will be matched in the Proteins dataframe and used to pull additional information about the peptide. 

In [3]:
pep['Accession'] = pep['Master Protein Accessions'].str.split('; ', expand=True)[0]
pro['Entrez'] = pro['Entrez Gene ID'].str.split('; ', expand=True)[0]
pro['Ensembl'] = pro['Ensembl Gene ID'].str.split('; ', expand=True)[0]

pep = pep.merge(pro[['Accession', 'Description', 'Entrez', 'Gene Symbol']], on='Accession')

Next, import a pre-generated dataframe (stored as a pickled binary format). The information is based on the MitoCarta2 database, pulled from MitoMiner (http://mitominer.mrc-mbu.cam.ac.uk/release-4.0/begin.do) with additional, manually curated information about the particular location within the mitochondria. This will be used to identify peptides and proteins as mitochondrial.

IMS = intermembrane space

matrix = mitochondrial matrix

In [4]:
mitocarta_df = pd.read_csv('mitocarta.csv')

pep['MitoCarta2_List'] = pep.Entrez.isin(mitocarta_df.MouseGeneID.astype(str))
pep['Matrix'] = pep.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.Matrix].astype(str))
pep['IMS'] = pep.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.IMS].astype(str))

pro['MitoCarta2_List'] = pro.Entrez.isin(mitocarta_df.MouseGeneID.astype(str))
pro['Matrix'] = pro.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.Matrix].astype(str))
pro['IMS'] = pro.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.IMS].astype(str))

Next, we will identify columns that we want to store in a "Master Index". This will contian key information we want to retain from the proteome discoverer 2.2 exported file

In [5]:
# a list to of columns to make a master index
protein_mi_list = ['Accession', 
                   'Description',
                   'Entrez',
                   'Gene Symbol',
                   'Ensembl',
                   'MitoCarta2_List',
                   'Matrix', 
                   'IMS',
                   'Coverage [%]',
                   '# Peptides',
                   '# PSMs',
                   '# Protein Unique Peptides',
                   '# Unique Peptides', 
                   'Chromosome',
                   'Modifications',
                   'Acetyl (K) Count',
                   'Acetyl (K) Positions']

peptide_mi_list = ['Accession', 
                   'Description',
                   'Entrez',
                   'Gene Symbol',
                   'Modifications', 
                   'MitoCarta2_List',
                   'Matrix', 
                   'IMS', 
                   'Modifications in Proteins',
                   'Modifications in Proteins (all Sites)',
                   'Positions in Proteins',]

Now, we identify the abundance columns from the experiment. These contain the quantification data for each TMT channel. There are 2 fractions that are mentioned:

- F1 = "Input", the unenriched, pooled TMT-labeled peptides for all 10 samples. 
- F2 = "Acetyl", the Acetyl-lysine IP-enriched fraction 

The numbers (126, 130C) indicate the specific TMT tag associated with the column. 

Categorical Variables:
- DFC = Dual flox control (Sirt3 and Crat floxed)
- DKO = Dual Knock-out (Sirt3 and Crat knockout via MCK-Cre)
- Sham = Surgical Controls for TACMI
- TAC = Trans-aortic constriction, a model of heart failure

In [6]:
input_abundance_list = ['Abundance: F1: 126, Sample, Input [Fraction], DKO [Genotype], Sham [Procedure]',
                        'Abundance: F1: 127N, Sample, Input [Fraction], DFC [Genotype], Sham [Procedure]',
                        'Abundance: F1: 127C, Sample, Input [Fraction], DFC [Genotype], TAC [Procedure]',
                        'Abundance: F1: 128N, Sample, Input [Fraction], DFC [Genotype], Sham [Procedure]',
                        'Abundance: F1: 128C, Sample, Input [Fraction], DKO [Genotype], TAC [Procedure]',
                        'Abundance: F1: 129N, Sample, Input [Fraction], DFC [Genotype], TAC [Procedure]',
                        'Abundance: F1: 129C, Sample, Input [Fraction], DKO [Genotype], TAC [Procedure]',
                        'Abundance: F1: 130N, Sample, Input [Fraction], DKO [Genotype], TAC [Procedure]',
                        'Abundance: F1: 130C, Sample, Input [Fraction], DFC [Genotype], TAC [Procedure]',
                        'Abundance: F1: 131N, Sample, Input [Fraction], DKO [Genotype], Sham [Procedure]',
                        'Abundance: F1: 131C, Sample, Input [Fraction], DFC [Genotype], Sham [Procedure]',]

acetyl_abundance_list = ['Abundance: F4: 126, Sample, Acetyl [Fraction], DKO [Genotype], Sham [Procedure]',
                         'Abundance: F4: 127N, Sample, Acetyl [Fraction], DFC [Genotype], Sham [Procedure]',
                         'Abundance: F4: 127C, Sample, Acetyl [Fraction], DFC [Genotype], TAC [Procedure]',
                         'Abundance: F4: 128N, Sample, Acetyl [Fraction], DFC [Genotype], Sham [Procedure]',
                         'Abundance: F4: 128C, Sample, Acetyl [Fraction], DKO [Genotype], TAC [Procedure]',
                         'Abundance: F4: 129N, Sample, Acetyl [Fraction], DFC [Genotype], TAC [Procedure]',
                         'Abundance: F4: 129C, Sample, Acetyl [Fraction], DKO [Genotype], TAC [Procedure]',
                         'Abundance: F4: 130N, Sample, Acetyl [Fraction], DKO [Genotype], TAC [Procedure]',
                         'Abundance: F4: 130C, Sample, Acetyl [Fraction], DFC [Genotype], TAC [Procedure]',
                         'Abundance: F4: 131N, Sample, Acetyl [Fraction], DKO [Genotype], Sham [Procedure]',
                         'Abundance: F4: 131C, Sample, Acetyl [Fraction], DFC [Genotype], Sham [Procedure]',]

These are shorter, more friendly versions of those identifier that will be used to rename the columns

In [7]:
input_consolidated_list = ['DKO,Sham,126,Input',
                           'DFC,Sham,127N,Input',
                           'DFC,TAC,127C,Input',
                           'DFC,Sham,128N,Input',
                           'DKO,TAC,128C,Input',
                           'DFC,TAC,129N,Input',
                           'DKO,TAC,129C,Input',
                           'DKO,TAC,130N,Input',
                           'DFC,TAC,130C,Input',
                           'DKO,Sham,131N,Input',
                           'DFC,Sham,131C,Input']

acetyl_consolidated_list = ['DKO,Sham,126,Acetyl',
                            'DFC,Sham,127N,Acetyl',
                            'DFC,TAC,127C,Acetyl',
                            'DFC,Sham,128N,Acetyl',
                            'DKO,TAC,128C,Acetyl',
                            'DFC,TAC,129N,Acetyl',
                            'DKO,TAC,129C,Acetyl',
                            'DKO,TAC,130N,Acetyl',
                            'DFC,TAC,130C,Acetyl',
                            'DKO,Sham,131N,Acetyl',
                            'DFC,Sham,131C,Acetyl']

In [8]:
acetyl_columns_df = pd.DataFrame.from_records([col.split(',') for col in acetyl_consolidated_list], 
                                              columns=['Genotype', 'Procedure', 'Channel', 'Fraction'])

input_columns_df = pd.DataFrame.from_records([col.split(',') for col in input_consolidated_list], 
                                              columns=['Genotype', 'Procedure', 'Channel', 'Fraction'])

acetyl_columns_df.set_index(acetyl_columns_df.columns.tolist(), inplace=True)
input_columns_df.set_index(input_columns_df.columns.tolist(), inplace=True)

## Locate the Acetyl Peptides (Kac)

We look for peptides that were quantified from the Acetyl fraction AND contain an identified actyl-lysine residue present. While the acetyl fraction is enriched for acetylated lysine residues, not all identified peptides within this fraction contain this post-translational modification. 

In [9]:
acetyl_df = pep[pep['Modifications'].str.contains('Acetyl')][acetyl_abundance_list].dropna()
acetyl_df.columns = acetyl_columns_df.index

# Flatten the column headers for working in R
acetyl_df.columns = ['_'.join(col).strip() for col in acetyl_df.columns.values]

In [10]:
# Define the input - i.e., everything identified in the non-enriched fraction
input_df = pep[input_abundance_list].dropna()
input_df.columns = input_columns_df.index

# Flatten the column headers for symmetry with acetyl
input_df.columns = ['_'.join(col).strip() for col in input_df.columns.values]

## Locate the protein abundances
- Use values obtained from the input fraction - these represent the total proteome
- Only select proteins that are called "Master Proteins" by proteome discoverer 2.2 wiht a FDR < 0.01

In [11]:
protein_df = (pro[(pro.Master=='IsMasterProtein')
                  &(pro['Exp. q-value: Combined']<=0.01)]
              [input_abundance_list]
              .dropna(how='all')
              .copy())

# copy the input headers and rename for proteins
protein_df.columns = input_df.columns
protein_df.rename(columns=lambda x: x.replace('Input', 'Proteins'), inplace=True)


# Normalize Data
- Correct for variations in total protein loading per kit
- use the mean of the sums of total peptide loading (input fraction) to generate a correction factor
- apply this correction factor to the acetyl peptides, 'input fraction', and proteins

## Calculate the scaling factor for each plex 

In [12]:
channel_sums = input_df.sum(axis=0)

In [13]:
print('Individual Channel, Loading relative to the mean')

channel_sums.divide(channel_sums.mean())

Individual Channel, Loading relative to the mean


DKO_Sham_126_Input     1.050456
DFC_Sham_127N_Input    0.981561
DFC_TAC_127C_Input     1.083244
DFC_Sham_128N_Input    0.878188
DKO_TAC_128C_Input     0.927371
DFC_TAC_129N_Input     0.960688
DKO_TAC_129C_Input     0.832689
DKO_TAC_130N_Input     1.107269
DFC_TAC_130C_Input     1.111127
DKO_Sham_131N_Input    0.944913
DFC_Sham_131C_Input    1.122494
dtype: float64

In [14]:
print('Scaling factor to adjust relative loading to 1')

scaling_factor = 1/channel_sums.divide(channel_sums.mean())
scaling_factor

Scaling factor to adjust relative loading to 1


DKO_Sham_126_Input     0.951967
DFC_Sham_127N_Input    1.018786
DFC_TAC_127C_Input     0.923153
DFC_Sham_128N_Input    1.138708
DKO_TAC_128C_Input     1.078317
DFC_TAC_129N_Input     1.040921
DKO_TAC_129C_Input     1.200929
DKO_TAC_130N_Input     0.903123
DFC_TAC_130C_Input     0.899987
DKO_Sham_131N_Input    1.058299
DFC_Sham_131C_Input    0.890873
dtype: float64

## Apply the Scaling Factor

In [15]:
input_norm = (input_df * scaling_factor)
input_norm.columns = [str(col) + '_norm' for col in input_norm.columns]
input_norm.sum()

DKO_Sham_126_Input_norm     7.481019e+06
DFC_Sham_127N_Input_norm    7.481019e+06
DFC_TAC_127C_Input_norm     7.481019e+06
DFC_Sham_128N_Input_norm    7.481019e+06
DKO_TAC_128C_Input_norm     7.481019e+06
DFC_TAC_129N_Input_norm     7.481019e+06
DKO_TAC_129C_Input_norm     7.481019e+06
DKO_TAC_130N_Input_norm     7.481019e+06
DFC_TAC_130C_Input_norm     7.481019e+06
DKO_Sham_131N_Input_norm    7.481019e+06
DFC_Sham_131C_Input_norm    7.481019e+06
dtype: float64

In [16]:
protein_norm = (protein_df * scaling_factor.rename(lambda x: x.replace('Input', 'Proteins')))
protein_norm.columns = [str(col) + '_norm' for col in protein_norm.columns]
protein_norm.sum()

DKO_Sham_126_Proteins_norm     5.755288e+06
DFC_Sham_127N_Proteins_norm    5.747786e+06
DFC_TAC_127C_Proteins_norm     5.668154e+06
DFC_Sham_128N_Proteins_norm    5.724267e+06
DKO_TAC_128C_Proteins_norm     5.688557e+06
DFC_TAC_129N_Proteins_norm     5.669434e+06
DKO_TAC_129C_Proteins_norm     5.677445e+06
DKO_TAC_130N_Proteins_norm     5.669717e+06
DFC_TAC_130C_Proteins_norm     5.650777e+06
DKO_Sham_131N_Proteins_norm    5.765681e+06
DFC_Sham_131C_Proteins_norm    5.732359e+06
dtype: float64

In [17]:
acetyl_norm = acetyl_df * scaling_factor.rename(lambda x: x.replace('Input', 'Acetyl'))
acetyl_norm.columns = [str(col) + '_norm' for col in acetyl_norm.columns]
acetyl_norm.sum()

DKO_Sham_126_Acetyl_norm     673459.973557
DFC_Sham_127N_Acetyl_norm    242747.293643
DFC_TAC_127C_Acetyl_norm     212815.445736
DFC_Sham_128N_Acetyl_norm    212207.017448
DKO_TAC_128C_Acetyl_norm     436367.120598
DFC_TAC_129N_Acetyl_norm     210247.624787
DKO_TAC_129C_Acetyl_norm     450322.341837
DKO_TAC_130N_Acetyl_norm     331552.341353
DFC_TAC_130C_Acetyl_norm     199342.466531
DKO_Sham_131N_Acetyl_norm    704116.489627
DFC_Sham_131C_Acetyl_norm    412378.727224
dtype: float64

# Calculate the "relative occupancy" for each plex
- This adjusts the acetyl peptide quantification based on changes in protein quantification. This corrects not for experimental loading, but for genetic changes that may occur between the genetic knock-out and the flox control.
- The relative occupancy corrections will be almost un-noticed in these animals, however. 

In [18]:
# convert the input-normalized data into the log2 space
# subtract the row mean from each column to center each row around 0 (in log 2)
acetyl_norm_log2 = np.log2(acetyl_norm).sub(np.log2(acetyl_norm).mean(axis=1), axis='index')
input_norm_log2 = np.log2(input_norm).sub(np.log2(input_norm).mean(axis=1), axis='index')
protein_norm_log2 = np.log2(protein_norm).sub(np.log2(protein_norm).mean(axis=1), axis='index')


# rename columns to save the information
acetyl_norm_log2.columns = [str(col) + '_log2' for col in acetyl_norm_log2.columns]
input_norm_log2.columns = [str(col) + '_log2' for col in input_norm_log2.columns]
protein_norm_log2.columns = [str(col) + '_log2' for col in protein_norm_log2.columns]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log2
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log2
  after removing the cwd from sys.path.


Combine the the "raw", load-normalized, and log2-transformed data into a single dataframe

In [19]:
acetyl = pd.concat([pep.loc[acetyl_norm.index][peptide_mi_list], 
                    acetyl_df, 
                    acetyl_norm, 
                    acetyl_norm_log2,], 
                   axis=1)

inputs = pd.concat([pep.loc[input_norm.index][peptide_mi_list], 
                    input_df, 
                    input_norm, 
                    input_norm_log2], 
                   axis=1)

protein = pd.concat([pro.loc[protein_df.index][protein_mi_list], 
                     protein_df, 
                     protein_norm, 
                     protein_norm_log2], 
                    axis=1)


## Calculate the RO
- Merge protein information along with acetyl information into a single dataframe
- The protein abundance information will be used to correc the acetyl data (the relative occupancy)

In [20]:
protein_merge_cols = protein.filter(regex='_log2').columns.tolist() + ['Accession']

In [21]:
merged_acetyl = acetyl.merge(protein[protein_merge_cols], 
                             on='Accession', 
                             how='left', 
                             validate='m:1')

The relative occupancy of the post-translational modification is calculated by subtracting changes in the protien abundance from the acetyl-peptide abundance. For example, if a PTM site is found to be increasing by 2 fold, but the protein abundnace is also increasing by 2 fold, then the relative occupancy would be 0. 

In [22]:
# calculate relative occupancy in acetyl peptides

relative_occupancy = (merged_acetyl.filter(regex='Acetyl_norm_log2')
                      .sub(merged_acetyl.filter(regex='Proteins_norm_log2')
                           # rename the protein column headers to enable easy dataframe subtraction
                           .rename(columns=lambda x: x.replace('Proteins', 'Acetyl'))))

relative_occupancy.rename(columns=lambda x: x+'_ro', inplace=True)

relative_occupancy.head()

,DKO_Sham_126_Acetyl_norm_log2_ro,DFC_Sham_127N_Acetyl_norm_log2_ro,DFC_TAC_127C_Acetyl_norm_log2_ro,DFC_Sham_128N_Acetyl_norm_log2_ro,DKO_TAC_128C_Acetyl_norm_log2_ro,DFC_TAC_129N_Acetyl_norm_log2_ro,DKO_TAC_129C_Acetyl_norm_log2_ro,DKO_TAC_130N_Acetyl_norm_log2_ro,DFC_TAC_130C_Acetyl_norm_log2_ro,DKO_Sham_131N_Acetyl_norm_log2_ro,DFC_Sham_131C_Acetyl_norm_log2_ro
0,-0.154750,-0.384628,-0.446949,-0.295440,0.294292,-0.504665,0.761702,0.215371,-0.875279,0.331100,1.059245
1,0.337560,-1.151819,-0.217556,-1.135259,0.110443,-0.886445,0.058382,-0.226050,-1.182244,0.880765,3.412223
2,-0.967698,-0.400803,0.058308,-0.142698,-0.627324,-0.573430,0.111890,-0.655765,-1.273027,0.304846,4.165702
3,-2.035343,-1.624763,0.167718,1.032514,0.370244,-0.844965,0.517107,0.271285,0.319972,-0.670951,2.497181
4,0.141024,0.103013,-0.587898,-0.413034,0.000574,-0.423486,-0.090507,-0.006432,-0.341178,-0.040621,1.658546


In [23]:
# stick the relative occupancy data onto the merged data
merged_acetyl = pd.concat([merged_acetyl, relative_occupancy], axis=1)

# drop the protein columns out of the data
merged_acetyl.drop(columns=merged_acetyl.filter(regex='_Protein').columns.tolist(), 
                   inplace=True)

# *limma* analysis

## Export Files for *limma* analysis

It is probably just as easy to do this entire analysis in R, with the limma component incorporated into the data processing. I'm not very experienced in R, so the disjointed analysis with R and python was faster for me. Exporting the data from Python into a CSV to import into R was the quickest method for my purposes. One could also try the rpy2 library (I tinkered with it, but found it to be too much effort for the payoff) or exporting the dataframes from pandas into a feather/arrow format.

In [24]:
acetyl_norm.to_csv('limma/limma_export_dkotac_kac.csv')

merged_acetyl.filter(regex='_ro$').to_csv('limma/limma_export_dkotac_RO_kac.csv')

protein.filter(regex='.+_norm$').to_csv('limma/limma_export_dkotac_proteins.csv')

## Read Files from *limma* analysis

In [25]:
kac_limma = pd.read_csv('limma/eb_fit_dkotac_kac.tsv', sep='\t')
kac_ro_limma = pd.read_csv('limma/eb_fit_dkotac_RO_kac.tsv', sep='\t')
prot_limma = pd.read_csv('limma/eb_fit_dkotac_proteins.tsv', sep='\t')

In [26]:
# merge back onto data
acetyl = pd.concat([acetyl, kac_limma], axis=1)

protein = pd.concat([protein, prot_limma], axis=1)

merged_acetyl = pd.concat([merged_acetyl, kac_ro_limma], axis=1)

# rename to somethign more fitting now
ro_acetyl = merged_acetyl

Rename the limma results to have more intuitive names

- `Coef` = Log2 Fold Change comparison between two groups
- `Res` = Significance. 
    - 0 = not significant
    - 1 = significant difference increasing
    - -1 = significant difference decreasing

In [27]:
limma_match = ['Coef\.', 'p\.value\.', 't\.', 'Res\.']
limma_replace = ['_Log2FC', '_p_value', '_t', '_significant']

for match, replace in zip(limma_match, limma_replace):
    acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    protein.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    ro_acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)

Pull out useful information from the Description field into separate columns. Although there is a Gene Symbol field present, it isn't always populated and isn't always the same

In [28]:
acetyl['Name'] = acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
acetyl['Symbol'] = acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

protein['Name'] = protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
protein['Symbol'] = protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

ro_acetyl['Name'] = ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
ro_acetyl['Symbol'] = ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

# reorder the columns so the information
acetyl = acetyl[acetyl.columns.tolist()[-2:] + acetyl.columns.tolist()[:-2]]
protein = protein[protein.columns.tolist()[-2:] + protein.columns.tolist()[:-2]]
ro_acetyl = ro_acetyl[ro_acetyl.columns.tolist()[-2:] + ro_acetyl.columns.tolist()[:-2]]

Sort the data

In [29]:
protein.sort_values('Acetyl (K) Count', ascending=False, inplace=True)
acetyl.sort_values('DKO_TACvsDFC_TAC_Log2FC', ascending=False, inplace=True)
ro_acetyl.sort_values('DKO_TACvsDFC_TAC_Log2FC', ascending=False, inplace=True)

# Export the Data

In [30]:
writer = pd.ExcelWriter('processed_files/Exp3_DKOTAC_11plex.xlsx', options={'strings_to_numbers': True})

ro_acetyl.to_excel(writer, sheet_name='Exp3_DKOTAC_RO_Kac', index=False)
acetyl.to_excel(writer, sheet_name='Exp3_DKOTAC_Kac', index=False)
protein.to_excel(writer, sheet_name='Exp3_DKOTAC_Protein', index=False)

writer.save()

In [31]:
ro_acetyl.to_csv('processed_files/Exp3_DKOTAC_RO_Kac.csv', index=False)
acetyl.to_csv('processed_files/Exp3_DKOTAC_Kac.csv', index=False)
protein.to_csv('processed_files/Exp3_DKOTAC_Protein.csv', index=False)

# Top 25 Fold Changes

This is one (simple) method to assess the severity of hyperacetylation. We'll only look at the relative occupancy corrected acetylpeptide abundances for this analysis

First, we will limit our analysis to the mitochondria as defined by the mitocarta2 dataset

In [32]:
df = ro_acetyl[ro_acetyl.MitoCarta2_List==True].copy()

Now, we will move out of the log2 space for more intuitive comparisons

In [33]:
df['DKO_TACvsDFC_TAC_FC'] = np.exp2(df.DKO_TACvsDFC_TAC_Log2FC)
df['DKO_ShamvsDFC_Sham_FC'] = np.exp2(df.DKO_ShamvsDFC_Sham_Log2FC)

Now, we want to pull out the specific lysine residues acetylated on each individual peptide. We want these residue positions related back to the master protein (i.e., not the position within the peptide). This isn't easily achieved in how the PD data is currently exported. Some peptides could correspond to multiple protein accessions, each with different lysine positions. All possible accessions and positions are listed in the peptide output - however, the order is arbitrary. Therefor, we need to iterate though the accessions listed and match them back to the master protein accession. 

In [34]:
kac_sites = df['Modifications in Proteins'].str.split('; (?!K)', expand=True, )
kac_sites.head()

,0,1,2,3,4,5,6,7,8,9,10
1442,Q8QZT1 1xAcetyl [K265(100)],None,None,None,None,None,None,None,None,None,None
626,O08756 1xAcetyl [K104(100)],A2AFQ2 1xAcetyl [K104(100)],Q99N15 1xAcetyl [K104(100)],None,None,None,None,None,None,None,None
1523,H3BKG7 1xAcetyl [K127(100)],P97807 1xAcetyl [K467(100)],P97807-2 1xAcetyl [K427(100)],None,None,None,None,None,None,None,None
894,D3Z7X0 1xAcetyl [K542(100)],Q8K370 1xAcetyl [K1056(100)],None,None,None,None,None,None,None,None,None
560,Q8BMS1 1xAcetyl [K406(100)],None,None,None,None,None,None,None,None,None,None


In the above data, the peptide at index 225 is linked to the master protein accession Q99N15, located in column 3. In other rows with multiple accessions, the master protein accession can be found in any of the 10 columns listed. Therefore, we will iterate through the columns of these sites to match them back to the original accession. The matches will be combined into a list, and the list will be transformed into a Series

In [35]:
# list to hold the sites associated with the same master protein accession
sites = []
for i in np.arange(0, kac_sites.columns[-1]+1):
    bool_mask = df.Accession == kac_sites[i].str.split(' ', expand=True)[0]
    try:
        temp_sites = kac_sites[i].loc[bool_mask].str.split('(?<!;)\s', expand=True)[2]
        sites.append(temp_sites)
    except:
        continue

In [36]:
# convert the list into a pandas series     
sites = pd.concat(sites)

# remove the percent confidence and brackets 
sites = sites.str.replace('\(\d{1,6}\)', '').str.replace('\[(.+)\]', '\g<1>')

sites.rename('Acetylated Residues', inplace=True)

sites.head(10)

1442    K265
894     K542
560     K406
739      K70
2046    K331
1945     K94
2177    K189
1533     K46
521     K259
1479    K282
Name: Acetylated Residues, dtype: object

Now, we add the specific sites back to the dataframe and prepare to export

In [37]:
# merge the residues back onto the dataframe
df = df.merge(sites.to_frame(), left_index=True, right_index=True)

In [38]:
export_cols = ['Accession', 'Name', 'Symbol', 'Gene Symbol', 'Entrez', 
               'Matrix', 'IMS', 'Acetylated Residues', 'DKO_TACvsDFC_TAC_FC', 'DKO_ShamvsDFC_Sham_FC']

# look at the top TACMI changing values
df[export_cols].sort_values('DKO_TACvsDFC_TAC_FC', ascending=False).head(25)


,Accession,Name,Symbol,Gene Symbol,Entrez,Matrix,IMS,Acetylated Residues,DKO_TACvsDFC_TAC_FC,DKO_ShamvsDFC_Sham_FC
1442,Q8QZT1,"Acetyl-CoA acetyltransferase, mitochondrial",Acat1,Acat1,110446,True,False,K265,160.642323,94.129170
626,Q99N15,17beta-hydroxysteroid dehydrogenase type 10/sh...,Hsd17b10,Hsd17b10,15108,True,False,K104,139.822514,145.321052
1523,P97807-2,"Isoform Cytoplasmic of Fumarate hydratase, mit...",Fh,Fh1,14194,True,False,K427,128.970548,17.079841
894,D3Z7X0,"Acyl-Coenzyme A dehydrogenase family, member 12",Acad12,Acad12,338350,False,False,K542,128.444352,511.305328
560,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K406,118.666042,77.718462
739,Q3ULD5,"Methylcrotonoyl-CoA carboxylase beta chain, mi...",Mccc2,Mccc2,78038,False,False,K70,107.057862,53.503575
2046,Q99LC5,"Electron transfer flavoprotein subunit alpha, ...",Etfa,Etfa,110842,True,False,K331,93.576630,143.428684
1945,P97450,"ATP synthase-coupling factor 6, mitochondrial",Atp5j,Atp5j,11957,False,True,K94,83.365383,83.507346
2177,Q99KR7,"Peptidyl-prolyl cis-trans isomerase F, mitocho...",Ppif,Ppif,105675,True,False,K189,66.521064,137.274193
1533,P03930,ATP synthase protein 8,Mtatp8,ATP8,17706,False,False,K46,60.052727,48.938592


In [39]:
writer = pd.ExcelWriter('processed_files/Exp3_DKOTAC_Top25.xlsx', options={'strings_to_numbers': True})

df[export_cols].sort_values('DKO_TACvsDFC_TAC_FC', ascending=False).head(25).to_excel(writer, sheet_name='Exp3_Top25_Kac_TAC_Comp', index=False)
df[export_cols].sort_values('DKO_ShamvsDFC_Sham_FC', ascending=False).head(25).to_excel(writer, sheet_name='Exp3_Top25_Kac_Sham_Comp', index=False)

writer.save()


df[export_cols].sort_values('DKO_TACvsDFC_TAC_FC', ascending=False).head(25).to_csv('processed_files/Exp3_Top25_Kac_TAC_Comp.csv', index=False)
df[export_cols].sort_values('DKO_ShamvsDFC_Sham_FC', ascending=False).head(25).to_csv('processed_files/Exp3_Top25_Kac_Sham_Comp.csv', index=False)
